# Big Data in Finance Final Project

In [1]:
# Load necessary packages
import pandas as pd
import numpy as np

In [2]:
# Adjust Jupyter Notebook display size
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [3]:
# Import data
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Data Preparation

### Description of Variables

- Id: A unique identifier associated with an application.
- Product_Info_1-7: A set of normalized variables relating to the product applied for
- Ins_Age: Normalized age of applicant
- Ht: Normalized height of applicant
- Wt: Normalized weight of applicant
- BMI: Normalized BMI of applicant
- Employment_Info_1-6: A set of normalized variables relating to the employment history of the applicant.
- InsuredInfo_1-7: A set of normalized variables providing information about the applicant.
- Insurance_History_1-9: A set of normalized variables relating to the insurance history of the applicant.
- Family_Hist_1-5: A set of normalized variables relating to the family history of the applicant.
- Medical_History_1-41: A set of normalized variables relating to the medical history of the applicant.
- Medical_Keyword_1-48: A set of dummy variables relating to the presence of/absence of a medical keyword being associated with the application.
- Response: This is the target variable, an ordinal variable relating to the final decision associated with an application

In [4]:
# Get first idea of dataset
train_data.head()

,Id,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response
0,2,1,D3,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1,0.0,3,NaN,1,2,6,3,1,2,1,1,1,3,1,0.000667,1,1,2,2,NaN,0.598039,NaN,0.526786,4.0,112,2,1,1,3,2,2,1,NaN,3,2,3,3,240.0,3,3,1,1,2,1,2,3,NaN,1,3,3,1,3,2,3,NaN,1,3,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
1,5,1,A1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3,0.0,2,0.0018,1,2,6,3,1,2,1,2,1,3,1,0.000133,1,3,2,2,0.188406,NaN,0.084507,NaN,5.0,412,2,1,1,3,2,2,1,NaN,3,2,3,3,0.0,1,3,1,1,2,1,2,3,NaN,1,3,3,1,3,2,3,NaN,3,1,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
2,6,1,E1,26,0.076923,2,3,1,0.029851,0.745455,0.288703,0.428780,0.030,9,1,0.0,2,0.0300,1,2,8,3,1,1,1,2,1,1,3,NaN,3,2,3,3,0.304348,NaN,0.225352,NaN,10.0,3,2,2,1,3,2,2,2,NaN,3,2,3,3,NaN,1,3,1,1,2,1,2,3,NaN,2,2,3,1,3,2,3,NaN,3,3,1,3,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
3,7,1,D4,10,0.487179,2,3,1,0.164179,0.672727,0.205021,0.352438,0.042,9,1,0.0,3,0.2000,2,2,8,3,1,2,1,2,1,1,3,NaN,3,2,3,3,0.420290,NaN,0.352113,NaN,0.0,350,2,2,1,3,2,2,2,NaN,3,2,3,3,NaN,1,3,1,1,2,2,2,3,NaN,1,3,3,1,3,2,3,NaN,3,3,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
4,8,1,D2,26,0.230769,2,3,1,0.417910,0.654545,0.234310,0.424046,0.027,9,1,0.0,2,0.0500,1,2,6,3,1,2,1,2,1,1,3,NaN,3,2,3,2,0.463768,NaN,0.408451,NaN,NaN,162,2,2,1,3,2,2,2,NaN,3,2,3,3,NaN,1,3,1,1,2,1,2,3,NaN,2,2,3,1,3,2,3,NaN,3,3,1,3,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8


In [5]:
# Isolate target variable
train_data_target = train_data.iloc[:,-1]
train_data = train_data.iloc[:,:-1]

test_data_target = test_data.iloc[:,-1]
test_data = test_data.iloc[:,:-1]

In [6]:
# Drop Id column (we don't want it as feature for the model)
train_data = train_data.iloc[:,1:]
test_data = test_data.iloc[:,1:]

In [7]:
# Display feature columns
train_data.columns

Index(['Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_4', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Ins_Age', 'Ht', 'Wt',
       ...
       'Medical_Keyword_39', 'Medical_Keyword_40', 'Medical_Keyword_41', 'Medical_Keyword_42', 'Medical_Keyword_43', 'Medical_Keyword_44', 'Medical_Keyword_45', 'Medical_Keyword_46', 'Medical_Keyword_47', 'Medical_Keyword_48'], dtype='object', length=126)

In [8]:
# Check summary stats
train_data.describe()

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
count,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59362.000000,59381.000000,59381.000000,52602.000000,59381.000000,48527.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,33985.000000,59381.000000,59381.000000,59381.000000,59381.000000,30725.000000,25140.000000,40197.000000,17570.000000,50492.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,557.000000,59381.000000,59381.000000,59381.000000,59381.000000,14785.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,3801.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,1107.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000,59381.000000
mean,1.026355,24.415655,0.328952,2.006955,2.673599,1.043583,0.405567,0.707283,0.292587,0.469462,0.077582,8.641821,1.300904,0.006283,2.142958,0.361469,1.209

In [9]:
# Show columns with missing data (test set)
train_data.isnull().sum().nlargest(10)

Medical_History_10     58824
Medical_History_32     58274
Medical_History_24     55580
Medical_History_15     44596
Family_Hist_5          41811
Family_Hist_3          34241
Family_Hist_2          28656
Insurance_History_5    25396
Family_Hist_4          19184
Employment_Info_6      10854
dtype: int64

In [10]:
# Show percentage of missing data (train set)
missing = train_data.isnull().sum()/len(train_data)
missing = missing[missing>0]
missing.sort_values(ascending=False)

Medical_History_10     0.990620
Medical_History_32     0.981358
Medical_History_24     0.935990
Medical_History_15     0.751015
Family_Hist_5          0.704114
Family_Hist_3          0.576632
Family_Hist_2          0.482579
Insurance_History_5    0.427679
Family_Hist_4          0.323066
Employment_Info_6      0.182786
Medical_History_1      0.149694
Employment_Info_4      0.114161
Employment_Info_1      0.000320
dtype: float64

In [11]:
# Show percentage of missing data (test set)
missing_test = test_data.isnull().sum()/len(test_data)
missing_test = missing_test[missing_test > 0]
missing_test.sort_values(ascending=False)

Medical_History_10     0.989831
Medical_History_32     0.982241
Medical_History_24     0.940299
Medical_History_15     0.752036
Family_Hist_5          0.689299
Family_Hist_3          0.559777
Family_Hist_2          0.499874
Insurance_History_5    0.410068
Family_Hist_4          0.337819
Employment_Info_6      0.191601
Medical_History_1      0.150367
Employment_Info_4      0.108120
Employment_Info_1      0.000152
dtype: float64

In [12]:
# Columns with missing values:
# Employment_Info_1, Employment_Info_4, Employment_Info_6, Insurance_History_5, Family_Hist_2, Family_Hist3
# Family_Hist_4, Family_Hist_5, Medical_History_1, Medical_History_10, Medical_History_15, Medical_History_24
# Medical_History_32
# We will treat them now

In [13]:
# Inspect data with missing values
train_data[missing.index].head()

,Employment_Info_1,Employment_Info_4,Employment_Info_6,Insurance_History_5,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_10,Medical_History_15,Medical_History_24,Medical_History_32
0,0.028,0.0,NaN,0.000667,NaN,0.598039,NaN,0.526786,4.0,NaN,240.0,NaN,NaN
1,0.000,0.0,0.0018,0.000133,0.188406,NaN,0.084507,NaN,5.0,NaN,0.0,NaN,NaN
2,0.030,0.0,0.0300,NaN,0.304348,NaN,0.225352,NaN,10.0,NaN,NaN,NaN,NaN
3,0.042,0.0,0.2000,NaN,0.420290,NaN,0.352113,NaN,0.0,NaN,NaN,NaN,NaN
4,0.027,0.0,0.0500,NaN,0.463768,NaN,0.408451,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# It seems as if the NaN values for Family_Hist_2 and Family_Hist_3 
# as well as Family_Hist_4 and Family_Hist_5 alternate

# Let's check whether there are any rows where both of them have a value
print(len(train_data[np.invert(train_data["Family_Hist_2"].isnull()) & np.invert(train_data["Family_Hist_3"].isnull())]))
print(len(train_data[np.invert(train_data["Family_Hist_4"].isnull()) & np.invert(train_data["Family_Hist_5"].isnull())]))
print(len(test_data[np.invert(test_data["Family_Hist_2"].isnull()) & np.invert(test_data["Family_Hist_3"].isnull())]))
print(len(test_data[np.invert(test_data["Family_Hist_4"].isnull()) & np.invert(test_data["Family_Hist_5"].isnull())]))

# There are no rows where this is the case

0
0
0
0


In [15]:
# Are there any rows where both are NaN?
print(len(train_data[train_data["Family_Hist_2"].isnull() & train_data["Family_Hist_3"].isnull()])/len(train_data))
print(len(train_data[train_data["Family_Hist_4"].isnull() & train_data["Family_Hist_5"].isnull()])/len(train_data))
print(len(test_data[test_data["Family_Hist_2"].isnull() & test_data["Family_Hist_3"].isnull()])/len(test_data))
print(len(test_data[test_data["Family_Hist_4"].isnull() & test_data["Family_Hist_5"].isnull()])/len(test_data))


# It is the case in ~6% or ~3% of the cases for both the test and train dataset

0.05921085869217427
0.027180411242653375
0.05965089805211232
0.02711864406779661


In [16]:
train_data[train_data["Family_Hist_5"]==0]

,Product_Info_1,Product_Info_2,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Family_Hist_2,Family_Hist_3,Family_Hist_4,Family_Hist_5,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_10,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_15,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_24,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_32,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48
35080,1,A3,26,0.076923,2,3,1,0.000000,0.672727,0.152720,0.257087,0.022,9,1,0.00,2,NaN,1,2,6,3,1,1,1,2,1,1,3,NaN,3,2,3,3,0.0,NaN,NaN,0.0,NaN,162,2,2,1,3,2,2,2,NaN,3,2,3,3,NaN,1,3,1,1,2,1,2,3,NaN,1,3,3,1,3,2,3,NaN,3,3,1,2,1,1,3,3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
56382,1,D2,26,0.230769,2,3,1,0.716418,0.781818,0.328452,0.456237,0.000,1,3,0.04,2,0.225,1,2,8,3,1,1,1,2,1,3,2,0.000267,1,3,2,3,NaN,0.578431,NaN,0.0,1.0,16,2,1,1,3,2,2,1,NaN,3,2,3,3,240.0,1,3,1,1,2,1,2,3,NaN,1,3,3,1,3,2,3,NaN,1,3,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
58155,1,D1,26,0.076923,2,3,1,0.805970,0.636364,0.225941,0.426893,0.051,1,3,0.00,2,0.260,1,2,6,3,1,2,1,2,1,3,1,0.000667,1,3,2,3,NaN,NaN,NaN,0.0,1.0,478,2,2,1,3,2,2,1,NaN,3,3,3,3,NaN,1,3,1,1,2,2,2,3,NaN,2,2,3,1,3,2,3,NaN,3,3,1,3,1,1,1,3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [17]:
# Firstly we decided to create new columns that combine the values of both variables
train_data["Family_Hist_2_3"] = np.nan_to_num(train_data["Family_Hist_2"]) + np.nan_to_num(train_data["Family_Hist_3"])
train_data["Family_Hist_4_5"] = np.nan_to_num(train_data["Family_Hist_4"]) + np.nan_to_num(train_data["Family_Hist_5"])
test_data["Family_Hist_2_3"] = np.nan_to_num(test_data["Family_Hist_2"]) + np.nan_to_num(test_data["Family_Hist_3"])
test_data["Family_Hist_4_5"] = np.nan_to_num(test_data["Family_Hist_4"]) + np.nan_to_num(test_data["Family_Hist_5"])

In [18]:
# As np.nan_to_num transforms nan to 0, 
# we need to set all rows where both family history values where nan back to nan
train_data["Family_Hist_2_3"].iloc[train_data[train_data["Family_Hist_2"].isnull() & train_data["Family_Hist_3"].isnull()].index] = np.nan
train_data["Family_Hist_4_5"].iloc[train_data[train_data["Family_Hist_4"].isnull() & train_data["Family_Hist_5"].isnull()].index] = np.nan
test_data["Family_Hist_2_3"].iloc[test_data[test_data["Family_Hist_2"].isnull() & test_data["Family_Hist_3"].isnull()].index] = np.nan
test_data["Family_Hist_4_5"].iloc[test_data[test_data["Family_Hist_4"].isnull() & test_data["Family_Hist_5"].isnull()].index] = np.nan

/Users/robertquick/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
# In a next step, to keep the original information of the values being split up,
# we introduce a dummy variable that takes the following values:
# In the case of Family_Hist_2_3....
# 0 if Family Hist 2 had a value
# 1 if Family Hist 3 had a value
# NaN if both had a NaN
# In the case of Family_Hist_4_5....
# 0 if Family Hist 4 had a value
# 1 if Family Hist 5 had a value
# NaN if both had a NaN
train_data["Dummy_Family_Hist_2_3"] = 0
train_data["Dummy_Family_Hist_2_3"].iloc[train_data[train_data["Family_Hist_2"].isnull()].index] = 1
train_data["Dummy_Family_Hist_2_3"].iloc[train_data[train_data["Family_Hist_2_3"].isnull() ].index] = np.nan
train_data["Dummy_Family_Hist_4_5"] = 0
train_data["Dummy_Family_Hist_4_5"].iloc[train_data[train_data["Family_Hist_4"].isnull()].index] = 1
train_data["Dummy_Family_Hist_4_5"].iloc[train_data[train_data["Family_Hist_4_5"].isnull() ].index] = np.nan

test_data["Dummy_Family_Hist_2_3"] = 0
test_data["Dummy_Family_Hist_2_3"].iloc[test_data[test_data["Family_Hist_2"].isnull()].index] = 1
test_data["Dummy_Family_Hist_2_3"].iloc[test_data[test_data["Family_Hist_2_3"].isnull() ].index] = np.nan
test_data["Dummy_Family_Hist_4_5"] = 0
test_data["Dummy_Family_Hist_4_5"].iloc[test_data[test_data["Family_Hist_4"].isnull()].index] = 1
test_data["Dummy_Family_Hist_4_5"].iloc[test_data[test_data["Family_Hist_4_5"].isnull() ].index] = np.nan

/Users/robertquick/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
# Drop columns with >50% missing values now, as well as redundant Family_Hist culumns
train_data = train_data.drop(['Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4','Family_Hist_5','Medical_History_10','Medical_History_15','Medical_History_24','Medical_History_32'], axis=1)
test_data = test_data.drop(['Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4','Family_Hist_5','Medical_History_10','Medical_History_15','Medical_History_24','Medical_History_32'], axis=1)

In [21]:
# Display unique value count for each column and type of each column
for column in train_data.columns:
    print("Type of column " + str(column) + " is: " + str(type(train_data[column][1])))
    print("Unique value count for column " + str(column) + " is: " + str(train_data[column].nunique()))
    print("Unique values of column " + str(column) + " are: " + str(train_data[column].unique()) + "\n")

Type of column Product_Info_1 is: <class 'numpy.int64'>
Unique value count for column Product_Info_1 is: 2
Unique values of column Product_Info_1 are: [1 2]

Type of column Product_Info_2 is: <class 'str'>
Unique value count for column Product_Info_2 is: 19
Unique values of column Product_Info_2 are: ['D3' 'A1' 'E1' 'D4' 'D2' 'A8' 'A2' 'D1' 'A7' 'A6' 'A3' 'A5' 'C4' 'C1' 'B2'
 'C3' 'C2' 'A4' 'B1']

Type of column Product_Info_3 is: <class 'numpy.int64'>
Unique value count for column Product_Info_3 is: 34
Unique values of column Product_Info_3 are: [10 26 21 31 29 37 15 36  8  4 30 11 23 28 13  9  6 33 32  3 18  2 22 17 38
 19 34 12 24 20  1 16  5 27]

Type of column Product_Info_4 is: <class 'numpy.float64'>
Unique value count for column Product_Info_4 is: 1491
Unique values of column Product_Info_4 are: [ 0.07692308  0.48717949  0.23076923 ...,  0.10993846  0.75393128
  0.10024923]

Type of column Product_Info_5 is: <class 'numpy.int64'>
Unique value count for column Product_Info_5 is:

In [22]:
# Product_Info_1, Product_Info_5, Product_Info6, Employment_Info_3, Employment_Info_5, InsuredInfo_2
# InsuredInfo_4, InsuredInfo_5, InsuredInfo_6, InsuredInfo_7, Insurance_History_1, Medical_History4
# Medical_History_22, Medical_History_33, Medical_History_38, Medical_Keyword_1-48
# are numeric categorical variables with 2 categories

# Product_Info_3, Product_Info_7, Employment_Info_2, InsuredInfo_1, InsuredInfo3, InsuranceHistory_2
# Insurance_History3, Insurance_History_4, Insurance_History_7, Insurance_History_8, Insurance_History_9
# Family_Hist_1, Medical_History_3, Medical_History_5, Medical_History_6, Medical_History_7, Medical_History_8
# Medical_History_9, Medical_History_11, Medical_History_12, Medical_History_13, Medical_History_14
# Medical_History_16, Medical_History_17, Medical_History_18, Medical_History_19, Medical_History_20
# Medical_History_21, Medical_History_23, Medical_History_25-31, Medical_History_34-37, Medical_History_39-41
# are numeric categorical variables with more than 2 categories

# Product_Info_4, Ins_Age, Ht, Wt, BMI, Employment_Info_1, Employment_Info_4, Employment_Info_6
# Insurance_History5, Family_Hist_2, Family_Hist_3, Familiy_Hist_4, Family_Hist_5, Medical_History_1
# Medical_History2, Medical_History_10, Medical_History_15, Medical_History_24, Medical_History_32
# are numeric continuous variables

# Response
# is the target variable with 8 numeric categories.

In [23]:
# We observe that Product_Info_2 is a categorical variable with 19 non-numeric categories
# Transform Product_Info_2 to dummy variable
train_data = pd.get_dummies(train_data, columns=['Product_Info_2'])
test_data = pd.get_dummies(test_data, columns=['Product_Info_2'])

In [24]:
train_data.head()

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Family_Hist_2_3,Family_Hist_4_5,Dummy_Family_Hist_2_3,Dummy_Family_Hist_4_5,Product_Info_2_A1,Product_Info_2_A2,Product_Info_2_A3,Product_Info_2_A4,Product_Info_2_A5,Product_Info_2_A6,Product_Info_2_A7,Product_Info_2_A8,Product_Info_2_B1,Product_Info_2_B2,Product_Info_2_C1,Product_Info_2_C2,Product_Info_2_C3,Product_Info_2_C4,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_E1
0,1,10,0.076923,2,1,1,0.641791,0.581818,0.148536,0.323008,0.028,12,1,0.0,3,NaN,1,2,6,3,1,2,1,1,1,3,1,0.000667,1,1,2,2,4.0,112,2,1,1,3,2,2,1,3,2,3,3,3,3,1,1,2,1,2,3,1,3,3,1,3,2,3,1,3,1,2,2,1,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.598039,0.526786,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,26,0.076923,2,3,1,0.059701,0.600000,0.131799,0.272288,0.000,1,3,0.0,2,0.0018,1,2,6,3,1,2,1,2,1,3,1,0.000133,1,3,2,2,5.0,412,2,1,1,3,2,2,1,3,2,3,3,1,3,1,1,2,1,2,3,1,3,3,1,3,2,3,3,1,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.188406,0.084507,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,26,0.076923,2,3,1,0.029851,0.745455,0.288703,0.428780,0.030,9,1,0.0,2,0.0300,1,2,8,3,1,1,1,2,1,1,3,NaN,3,2,3,3,10.0,3,2,2,1,3,2,2,2,3,2,3,3,1,3,1,1,2,1,2,3,2,2,3,1,3,2,3,3,3,1,3,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.304348,0.225352,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,10,0.487179,2,3,1,0.164179,0.672727,0.205021,0.352438,0.042,9,1,0.0,3,0.2000,2,2,8,3,1,2,1,2,1,1,3,NaN,3,2,3,3,0.0,350,2,2,1,3,2,2,2,3,2,3,3,1,3,1,1,2,2,2,3,1,3,3,1,3,2,3,3,3,1,2,2,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.420290,0.352113,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1,26,0.230769,2,3,1,0.417910,0.654545,0.234310,0.424046,0.027,9,1,0.0,2,0.0

In [25]:
# In order to deal with missing values we will use KNN imputation
# This usually performs better than simple imputation with statistical summary statistics such as the
# mean or median

# KNN is an algorithm that is useful for matching a point with its closest k neighbors
# in a multi-dimensional space. It can be used for data that are continuous, discrete, ordinal 
# and categorical which makes it particularly useful for dealing with all kind of missing data.
# The assumption behind using KNN for missing values is that a point value can be approximated by 
# the values of the points that are closest to it, based on other variables.

# Save column names
column_names = np.asarray(train_data.columns)

from fancyimpute import KNN,SimpleFill
#train_data_imputed = KNN(k=5).complete(train_data)
train_data_imputed = SimpleFill(fill_method='mean').complete(train_data)

Using TensorFlow backend.
/Users/robertquick/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [26]:
#Convert back to dataframe
train_data_complete = pd.DataFrame(train_data_imputed, columns = column_names)
train_data_complete.head()

### do we need to convert back to correct type of column (integer etc.)?

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Family_Hist_2_3,Family_Hist_4_5,Dummy_Family_Hist_2_3,Dummy_Family_Hist_4_5,Product_Info_2_A1,Product_Info_2_A2,Product_Info_2_A3,Product_Info_2_A4,Product_Info_2_A5,Product_Info_2_A6,Product_Info_2_A7,Product_Info_2_A8,Product_Info_2_B1,Product_Info_2_B2,Product_Info_2_C1,Product_Info_2_C2,Product_Info_2_C3,Product_Info_2_C4,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_E1
0,1.0,10.0,0.076923,2.0,1.0,1.0,0.641791,0.581818,0.148536,0.323008,0.028,12.0,1.0,0.0,3.0,0.361469,1.0,2.0,6.0,3.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,0.000667,1.0,1.0,2.0,2.0,4.000000,112.0,2.0,1.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,3.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,3.0,3.0,1.0,3.0,2.0,3.0,1.0,3.0,1.0,2.0,2.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.598039,0.526786,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,26.0,0.076923,2.0,3.0,1.0,0.059701,0.600000,0.131799,0.272288,0.000,1.0,3.0,0.0,2.0,0.001800,1.0,2.0,6.0,3.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,0.000133,1.0,3.0,2.0,2.0,5.000000,412.0,2.0,1.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,3.0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,1.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.188406,0.084507,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,26.0,0.076923,2.0,3.0,1.0,0.029851,0.745455,0.288703,0.428780,0.030,9.0,1.0,0.0,2.0,0.030000,1.0,2.0,8.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,0.001733,3.0,2.0,3.0,3.0,10.000000,3.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0

In [27]:
#KNN imputation for test set

#Save column names
test_column_names = np.asarray(test_data.columns)

#Complete missing values
# test_data_imputed = KNN(k=5).complete(test_data)
test_data_imputed = SimpleFill(fill_method='mean').complete(test_data)

In [28]:
#Convert back to dataframe
test_data_complete = pd.DataFrame(test_data_imputed, columns = test_column_names)
test_data_complete.head()

### do we need to convert back to correct type of column (integer etc.)?

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Family_Hist_2_3,Family_Hist_4_5,Dummy_Family_Hist_2_3,Dummy_Family_Hist_4_5,Product_Info_2_A1,Product_Info_2_A2,Product_Info_2_A3,Product_Info_2_A4,Product_Info_2_A5,Product_Info_2_A6,Product_Info_2_A7,Product_Info_2_A8,Product_Info_2_B1,Product_Info_2_B2,Product_Info_2_C1,Product_Info_2_C2,Product_Info_2_C3,Product_Info_2_C4,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_E1
0,1.0,26.0,0.487179,2.0,3.0,1.0,0.611940,0.781818,0.338912,0.472262,0.150,3.0,1.0,0.00,2.0,0.50,2.0,2.0,11.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,0.001642,3.0,2.0,3.0,3.0,2.000000,16.0,2.0,2.0,1.0,3.0,1.0,2.0,2.0,3.0,2.0,1.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,2.0,3.0,3.0,3.0,1.0,3.0,2.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.627451,0.760563,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,26.0,0.076923,2.0,3.0,1.0,0.626866,0.727273,0.311715,0.484984,0.000,1.0,3.0,0.07,2.0,0.20,1.0,2.0,8.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,0.001667,1.0,1.0,2.0,2.0,5.000000,261.0,3.0,1.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,3.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0,3.0,1.0,3.0,2.0,1.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.529412,0.746479,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,26.0,0.144667,2.0,3.0,1.0,0.582090,0.709091,0.320084,0.519103,0.143,9.0,1.0,0.00,2.0,0.45,1.0,2.0,3.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,0.001642,3.0,2.0,3.0,3.0,3.000000,132.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,2.0,3.0,1.0,1.0,2.0,3.0,1.0,3.0,1.0

In [29]:
# Feature extraction: Build sum of Medical_Keyword dummies
train_data_complete["Medical_Keyword_Sum"] = train_data_complete.iloc[:,-48:].sum(axis=1)
test_data_complete["Medical_Keyword_Sum"] = test_data_complete.iloc[:,-48:].sum(axis=1)

In [30]:
train_data_complete.head()

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Family_Hist_2_3,Family_Hist_4_5,Dummy_Family_Hist_2_3,Dummy_Family_Hist_4_5,Product_Info_2_A1,Product_Info_2_A2,Product_Info_2_A3,Product_Info_2_A4,Product_Info_2_A5,Product_Info_2_A6,Product_Info_2_A7,Product_Info_2_A8,Product_Info_2_B1,Product_Info_2_B2,Product_Info_2_C1,Product_Info_2_C2,Product_Info_2_C3,Product_Info_2_C4,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_E1,Medical_Keyword_Sum
0,1.0,10.0,0.076923,2.0,1.0,1.0,0.641791,0.581818,0.148536,0.323008,0.028,12.0,1.0,0.0,3.0,0.361469,1.0,2.0,6.0,3.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,0.000667,1.0,1.0,2.0,2.0,4.000000,112.0,2.0,1.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,3.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,3.0,3.0,1.0,3.0,2.0,3.0,1.0,3.0,1.0,2.0,2.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.598039,0.526786,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.124825
1,1.0,26.0,0.076923,2.0,3.0,1.0,0.059701,0.600000,0.131799,0.272288,0.000,1.0,3.0,0.0,2.0,0.001800,1.0,2.0,6.0,3.0,1.0,2.0,1.0,2.0,1.0,3.0,1.0,0.000133,1.0,3.0,2.0,2.0,5.000000,412.0,2.0,1.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,3.0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,1.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.188406,0.084507,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.272913
2,1.0,26.0,0.076923,2.0,3.0,1.0,0.029851,0.745455,0.288703,0.428780,0.030,9.0,1.0,0.0,2.0,0.030000,1.0,2.0,8.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,0.001733,3.0,2.0,3.0,3.0,10.000000,3.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,3.0,3

In [31]:
test_data_complete.head()

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,Ht,Wt,BMI,Employment_Info_1,Employment_Info_2,Employment_Info_3,Employment_Info_4,Employment_Info_5,Employment_Info_6,InsuredInfo_1,InsuredInfo_2,InsuredInfo_3,InsuredInfo_4,InsuredInfo_5,InsuredInfo_6,InsuredInfo_7,Insurance_History_1,Insurance_History_2,Insurance_History_3,Insurance_History_4,Insurance_History_5,Insurance_History_7,Insurance_History_8,Insurance_History_9,Family_Hist_1,Medical_History_1,Medical_History_2,Medical_History_3,Medical_History_4,Medical_History_5,Medical_History_6,Medical_History_7,Medical_History_8,Medical_History_9,Medical_History_11,Medical_History_12,Medical_History_13,Medical_History_14,Medical_History_16,Medical_History_17,Medical_History_18,Medical_History_19,Medical_History_20,Medical_History_21,Medical_History_22,Medical_History_23,Medical_History_25,Medical_History_26,Medical_History_27,Medical_History_28,Medical_History_29,Medical_History_30,Medical_History_31,Medical_History_33,Medical_History_34,Medical_History_35,Medical_History_36,Medical_History_37,Medical_History_38,Medical_History_39,Medical_History_40,Medical_History_41,Medical_Keyword_1,Medical_Keyword_2,Medical_Keyword_3,Medical_Keyword_4,Medical_Keyword_5,Medical_Keyword_6,Medical_Keyword_7,Medical_Keyword_8,Medical_Keyword_9,Medical_Keyword_10,Medical_Keyword_11,Medical_Keyword_12,Medical_Keyword_13,Medical_Keyword_14,Medical_Keyword_15,Medical_Keyword_16,Medical_Keyword_17,Medical_Keyword_18,Medical_Keyword_19,Medical_Keyword_20,Medical_Keyword_21,Medical_Keyword_22,Medical_Keyword_23,Medical_Keyword_24,Medical_Keyword_25,Medical_Keyword_26,Medical_Keyword_27,Medical_Keyword_28,Medical_Keyword_29,Medical_Keyword_30,Medical_Keyword_31,Medical_Keyword_32,Medical_Keyword_33,Medical_Keyword_34,Medical_Keyword_35,Medical_Keyword_36,Medical_Keyword_37,Medical_Keyword_38,Medical_Keyword_39,Medical_Keyword_40,Medical_Keyword_41,Medical_Keyword_42,Medical_Keyword_43,Medical_Keyword_44,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Family_Hist_2_3,Family_Hist_4_5,Dummy_Family_Hist_2_3,Dummy_Family_Hist_4_5,Product_Info_2_A1,Product_Info_2_A2,Product_Info_2_A3,Product_Info_2_A4,Product_Info_2_A5,Product_Info_2_A6,Product_Info_2_A7,Product_Info_2_A8,Product_Info_2_B1,Product_Info_2_B2,Product_Info_2_C1,Product_Info_2_C2,Product_Info_2_C3,Product_Info_2_C4,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_E1,Medical_Keyword_Sum
0,1.0,26.0,0.487179,2.0,3.0,1.0,0.611940,0.781818,0.338912,0.472262,0.150,3.0,1.0,0.00,2.0,0.50,2.0,2.0,11.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,0.001642,3.0,2.0,3.0,3.0,2.000000,16.0,2.0,2.0,1.0,3.0,1.0,2.0,2.0,3.0,2.0,1.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,2.0,3.0,3.0,3.0,1.0,3.0,2.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.627451,0.760563,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.388014
1,1.0,26.0,0.076923,2.0,3.0,1.0,0.626866,0.727273,0.311715,0.484984,0.000,1.0,3.0,0.07,2.0,0.20,1.0,2.0,8.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,0.001667,1.0,1.0,2.0,2.0,5.000000,261.0,3.0,1.0,1.0,3.0,2.0,2.0,1.0,3.0,2.0,3.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,2.0,3.0,1.0,3.0,2.0,3.0,3.0,3.0,1.0,3.0,2.0,1.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.529412,0.746479,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.275891
2,1.0,26.0,0.144667,2.0,3.0,1.0,0.582090,0.709091,0.320084,0.519103,0.143,9.0,1.0,0.00,2.0,0.45,1.0,2.0,3.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,3.0,0.001642,3.0,2.0,3.0,3.0,3.000000,132.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,2.0,1.0,2.0,3.0,2

In [32]:
# Transform Product_Info_2 to categorical

### doesn't apply because product_info_2 column was dropped before

#train_data["Product_Info_2"] = train_data["Product_Info_2"].astype('category')
#train_data["Product_Info_2"] = train_data["Product_Info_2"].cat.codes

In [33]:
#Export cleaned data
train_data_complete.to_csv('cleaned_train_data.csv')
test_data_complete.to_csv('cleaned_test_data.csv')

# Modelling

## Multiclass Logistic Regression

In [ ]:
X = train_data_complete
y = train_data_target

In [ ]:
from sklearn import linear_model
clf = sklearn.linear_model.LogisticRegressionCV(class_weight='balanced', multi_class='multinomial', solver='lbfgs')
clf.fit(X, y)
pred = clf.predict(pred)

In [ ]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X, y) 
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
dec = clf.decision_function([[1]])
dec.shape[1]
clf.decision_function_shape = "ovr"
dec = clf.decision_function([[1]])
dec.shape[1]